In [1]:
import os
import time
import numpy as np
import pandas as pd

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("benstaf/nasdaq_2013_2023", data_files="train_data_deepseek_sentiment_2013_2018.csv")
# train = pd.DataFrame(dataset['train'])

In [ ]:
# chunks = pd.read_csv("../Task_1_FinRL_DeepSeek_Stock/data/nasdaq_exteral_data.csv", chunksize=1000, header=0)
# from itertools import islice
# for num, chunk in enumerate(islice(chunks, 20)):  # 처음 10개 chunk만 반복
#     print(chunk.Date, chunk.Stock_symbol)

0      2023-12-16 23:00:00 UTC
1      2023-12-12 00:00:00 UTC
2      2023-12-12 00:00:00 UTC
3      2023-12-07 00:00:00 UTC
4      2023-12-07 00:00:00 UTC
                ...           
995    2021-08-23 00:00:00 UTC
996    2021-08-22 00:00:00 UTC
997    2021-08-20 00:00:00 UTC
998    2021-08-20 00:00:00 UTC
999    2021-08-20 00:00:00 UTC
Name: Date, Length: 1000, dtype: object 0       A
1       A
2       A
3       A
4       A
       ..
995    AA
996    AA
997    AA
998    AA
999    AA
Name: Stock_symbol, Length: 1000, dtype: object
1000    2021-08-19 00:00:00 UTC
1001    2021-08-12 00:00:00 UTC
1002    2021-08-06 00:00:00 UTC
1003    2021-08-06 00:00:00 UTC
1004    2021-08-05 00:00:00 UTC
                 ...           
1995    2015-06-25 00:00:00 UTC
1996    2015-06-23 00:00:00 UTC
1997    2015-06-04 00:00:00 UTC
1998    2015-03-26 00:00:00 UTC
1999    2015-03-24 00:00:00 UTC
Name: Date, Length: 1000, dtype: object 1000      AA
1001      AA
1002      AA
1003      AA
1004      AA
    

In [2]:
import pandas as pd
tot_data = pd.read_csv("../Task_1_FinRL_DeepSeek_Stock/data/processed_data_v2.csv")

In [6]:
tot_data["tic"].unique()

array(['AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'ALGN', 'AMAT', 'AMD',
       'AMGN', 'AMZN', 'ANSS', 'ASML', 'AVGO', 'AZN', 'BIIB', 'BKNG',
       'BKR', 'CDNS', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CSCO', 'CSGP',
       'CSX', 'CTAS', 'CTSH', 'DLTR', 'DXCM', 'EA', 'EBAY', 'ENPH', 'EXC',
       'FANG', 'FAST', 'FTNT', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX',
       'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KLAC', 'LRCX', 'LULU',
       'MAR', 'MCHP', 'MDLZ', 'MELI', 'META', 'MNST', 'MRVL', 'MSFT',
       'MU', 'NFLX', 'NVDA', 'NXPI', 'ODFL', 'ON', 'ORLY', 'PANW', 'PAYX',
       'PCAR', 'PEP', 'QCOM', 'REGN', 'ROST', 'SBUX', 'SIRI', 'SNPS',
       'TMUS', 'TSLA', 'TXN', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL'],
      dtype=object)

# minmax scaler & Train, Test Split

In [8]:
TRAIN_START_DATE = '2013-01-01'
TRAIN_END_DATE = '2018-12-31'
TRADE_START_DATE = '2019-01-01'
TRADE_END_DATE = '2023-12-31'

In [29]:
def data_split(df, start, end, target_date_col="date"):
    """
    split the dataset into training or testing using date
    :param data: (df) pandas dataframe, start, end
    :return: (df) pandas dataframe
    """
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    # data.index = data[target_date_col].factorize()[0]
    return data

In [30]:
train = data_split(tot_data, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(tot_data, TRADE_START_DATE,TRADE_END_DATE)

In [31]:
from sklearn.preprocessing import MinMaxScaler

In [32]:
def min_max_data(train_data, test_data):
    """
    변수에 대한 min-max scaling을 수행하는 함수입니다.
    
    Args:
        train_data : 학습 데이터
        test_data : 테스트 데이터
    """
    delist_col= ['date', 'tic', 'gtaa_buy_signal', 'paa_buy_signal', 'daa_buy_signal', 'momentum_product', 'dm_buy_signal']
    scaler = MinMaxScaler()
    scaler.fit(train_data.drop(columns=delist_col))
    
    train_data_scaled = scaler.transform(train_data.drop(columns=delist_col))
    test_dataset_scaled = scaler.transform(test_data.drop(columns=delist_col))
    train_data_scaled = pd.DataFrame(train_data_scaled, columns=train_data.drop(columns=delist_col).columns)
    test_dataset_scaled = pd.DataFrame(test_dataset_scaled, columns=test_data.drop(columns=delist_col).columns)
    
    
    # TICKER, weekday를 다시 추가합니다.
    for col in delist_col:
        
        train_data_scaled[col] = train_data[col].values
        test_dataset_scaled[col] = test_data[col].values
    # train_data_scaled["TICKER"] = train_data["TICKER"].values
    # train_data_scaled["weekday"] = train_data["weekday"].values
    # test_dataset_scaled["TICKER"] = test_data["TICKER"].values
    # test_dataset_scaled["weekday"] = test_data["weekday"].values
    
    train_data_scaled.index = train_data.index
    test_dataset_scaled.index = test_data.index
    
    return train_data_scaled, test_dataset_scaled, scaler


In [33]:
train_scaled, test_scaled, scaler = min_max_data(train, trade)


In [ ]:
train.to_csv("./data/train.csv")
trade.to_csv("./data/test.csv")
train_scaled.to_csv("./data/train_scaled.csv")
test_scaled.to_csv("./data/test_scaled.csv")

In [2]:
train = pd.read_csv("./data/train.csv")
trade = pd.read_csv("./data/test.csv")
train_scaled = pd.read_csv("./data/train_scaled.csv")
test_scaled = pd.read_csv("./data/test_scaled.csv")

## Tensor 만들기

In [3]:
window_size = 20
asset_dim = train["tic"].nunique()
# feature_dim = len(features)


In [4]:
def _make_data_chaching(real_data):
    """
    데이터를 티커 별로 df대신 딕셔너리로 저장해서 속도를 올리기 위함
    """
    asset_data_dict = {}
    for asset in real_data["tic"].unique():
        asset_data_dict[asset] = real_data[real_data["tic"] == asset]
    return asset_data_dict

In [5]:
# rebalance_window = 22

def main_save_tensor(data, window_size):
    features = ['close', 'high', 'low', 'open', 'volume', 'mom_12m', 'mom_score', 'dm_score', 'macd', 'boll_ub',
        'boll_lb', 'rsi', 'cci', 'paa_score']
    imp = data.reset_index()
    date_index = imp["date"].unique()
    # asset_dim = real_train_data["TICKER"].nunique()
    # feature_dim = len(features)
    # 데이터 캐싱
    asset_data_dict = _make_data_chaching(imp)
    
    
    save_numpy_rebal_dict = {}
    for idx, day in enumerate(date_index):
        rebalance_date = date_index[idx-window_size+1:idx+1]
        if len(rebalance_date) < window_size:
            rebalance_date = date_index[:idx+1]
        total_li = []
        
        for date in rebalance_date:
            save_list = []
            for asset, asset_data in asset_data_dict.items():
                date_data = asset_data[asset_data["date"] == date]
                if not date_data.empty:
                    imp_1 = date_data[features].values
                    save_list.append(imp_1.reshape(-1))
                else:
                    # 데이터가 없으면 0으로 채움
                    save_list.append(np.zeros(len(features)))
            total_li.append(save_list)
        if len(total_li) < window_size:
            padding = [np.zeros(len(features)) for _ in range(len(asset_data_dict))]
            for _ in range(window_size - len(total_li)):
                total_li.insert(0, padding)
        save_numpy_rebal_dict[rebalance_date[-1]] = np.array(total_li)
    return save_numpy_rebal_dict
    


In [6]:
def main_test_tensor(data, window_size):
    features = ['close', 'high', 'low', 'open', 'volume', 'mom_12m', 'mom_score', 'dm_score', 'macd', 'boll_ub',
        'boll_lb', 'rsi', 'cci', 'paa_score']
    imp = data.reset_index()
    date_index = imp["date"].unique()
    # rolling_idx = 0
    # asset_dim = real_train_data["TICKER"].nunique()
    # feature_dim = len(features)
    # 데이터 캐싱
    asset_data_dict = _make_data_chaching(imp)


    save_numpy_rebal_dict = {}

    for idx, day in enumerate(date_index):
        rebalance_date = date_index[idx-window_size+1:idx+1]
        if len(rebalance_date) < window_size:
            continue
        # 윈도우 내 데이터를 저장할 리스트
        total_li = []

        for date in rebalance_date:
            save_list = []
            for asset, asset_data in asset_data_dict.items():
                # 캐싱된 데이터를 사용하여 연산
                date_data = asset_data[asset_data["date"] == date]
                if not date_data.empty:
                    imp_1 = date_data[features].values
                    save_list.append(imp_1.reshape(-1))
                else:
                    # 데이터가 없으면 0으로 채움
                    save_list.append(np.zeros(len(features)))
            
            total_li.append(save_list)
        if len(total_li) < window_size:
            padding = [np.zeros(len(features)) for _ in range(len(asset_data_dict))]
            for _ in range(window_size - len(total_li)):
                total_li.insert(0, padding)
        save_numpy_rebal_dict[rebalance_date[-1]] = total_li
        
    return save_numpy_rebal_dict
    

In [7]:
tot_data = pd.concat([train_scaled, test_scaled])

In [8]:
tot_data.set_index("date", inplace=True)

In [ ]:
tot_data = tot_data[tot_data.index >= "2018-11-01")]

In [10]:
save_test_dict = main_test_tensor(tot_data, window_size)

In [11]:
save_sub_episode_dict = main_save_tensor(train_scaled, window_size)

In [12]:
save_test_dict.keys()

dict_keys(['2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15', '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21', '2019-03-22', '2019-03-25', '2019-03-26', '2019-03-27', '2019-03-28', '2019-03-29', '2019-04-01', '2019-04-02', '2019-04-03', '2019-04-04', '2019-04-05', '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11', '2019-04-12', '2019-04-

In [14]:
save_sub_episode_dict.keys()

dict_keys(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11', '2013-01-14', '2013-01-15', '2013-01-16', '2013-01-17', '2013-01-18', '2013-01-22', '2013-01-23', '2013-01-24', '2013-01-25', '2013-01-28', '2013-01-29', '2013-01-30', '2013-01-31', '2013-02-01', '2013-02-04', '2013-02-05', '2013-02-06', '2013-02-07', '2013-02-08', '2013-02-11', '2013-02-12', '2013-02-13', '2013-02-14', '2013-02-15', '2013-02-19', '2013-02-20', '2013-02-21', '2013-02-22', '2013-02-25', '2013-02-26', '2013-02-27', '2013-02-28', '2013-03-01', '2013-03-04', '2013-03-05', '2013-03-06', '2013-03-07', '2013-03-08', '2013-03-11', '2013-03-12', '2013-03-13', '2013-03-14', '2013-03-15', '2013-03-18', '2013-03-19', '2013-03-20', '2013-03-21', '2013-03-22', '2013-03-25', '2013-03-26', '2013-03-27', '2013-03-28', '2013-04-01', '2013-04-02', '2013-04-03', '2013-04-04', '2013-04-05', '2013-04-08', '2013-04-09', '2013-04-10', '2013-04-11', '2013-04-12', '2013-04-

In [17]:
import pickle

In [18]:
with open("./data/test_daily.pkl", "wb") as f:
    pickle.dump(save_test_dict, f)
    
# save_sub_episode_dict = main_save_tensor(train_scaled, window_size)


with open("./data/train_daily.pkl", "wb") as f:
    pickle.dump(save_sub_episode_dict, f)

In [24]:
import torch

In [26]:
def make_subepisode_tensor(data):
    values = []
    for key, value in data.items():
        daily_tensor = []
        for daily_data in value:
            window_tensor = []
            for feature_data in daily_data:
                feature_tensor = torch.tensor(feature_data)
                window_tensor.append(feature_tensor)
            daily_window_tensor = torch.stack(window_tensor)
            daily_tensor.append(daily_window_tensor)
        total_tensor = torch.stack(daily_tensor)
        values.append(total_tensor)
    final_tensor = torch.stack(values)
    return final_tensor
    

def save_all_episodes(episodes, file_name):
    # 리스트를 사전 형태로 저장 (에피소드별 인덱스 포함)
    # data = {str(i): episode for i, episode in enumerate(episodes)}
    torch.save(episodes, file_name)
    print(f"All episodes saved to: {file_name}")
    return episodes


def main_tensor_make(data, file_name):
    episodes = make_subepisode_tensor(data)
    data = save_all_episodes(episodes, file_name)
    return data

In [27]:
episodes = main_tensor_make(save_sub_episode_dict, "./data/daily_data.pt")

All episodes saved to: ./data/daily_data.pt


In [28]:
test_episodes = main_tensor_make(save_test_dict, "./data/test_daily_data.pt")

All episodes saved to: ./data/test_daily_data.pt


In [30]:
episodes.shape

torch.Size([1509, 20, 84, 14])

In [31]:
test_episodes.shape

torch.Size([1257, 20, 84, 14])

In [33]:
len(save_sub_episode_dict.keys())

1509

## 환경 적용하기

## Backtesting 확인하기

In [6]:
pd.read_csv("../Task_1_FinRL_DeepSeek_Stock/data/train_data_deepseek_sentiment_2013_2018.csv")

,Unnamed: 0.1,Unnamed: 0,date,tic,close,high,low,open,volume,day,...,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,llm_sentiment
0,0,0,2013-01-02,AAPL,16.687340,19.821428,19.343929,19.779285,560518000.0,2.0,...,16.879896,16.284159,0.000000,-66.666667,100.000000,16.687340,16.687340,14.68,0.000000,NaN
1,1,1,2013-01-02,ADBE,38.340000,38.730000,37.919998,37.919998,6483800.0,2.0,...,16.879896,16.284159,0.000000,-66.666667,100.000000,38.340000,38.340000,14.68,0.000000,NaN
2,2,2,2013-01-02,ADI,33.714191,44.009998,42.799999,42.799999,3761100.0,2.0,...,16.879896,16.284159,0.000000,-66.666667,100.000000,33.714191,33.714191,14.68,0.000000,3.0
3,3,3,2013-01-02,ADP,39.267780,51.404739,50.702370,50.974541,3217561.0,2.0,...,16.879896,16.284159,0.000000,-66.666667,100.000000,39.267780,39.267780,14.68,0.000000,NaN
4,4,4,2013-01-02,ADSK,36.939999,36.950001,36.049999,36.049999,2998800.0,2.0,...,16.879896,16.284159,0.000000,-66.666667,100.000000,36.939999,36.939999,14.68,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126751,126751,126751,2018-12-28,VRTX,161.419998,164.839996,160.380005,163.520004,966700.0,4.0,...,187.585850,150.832147,44.935760,-57.015038,17.249474,168.107999,172.781499,28.34,53.769986,NaN
126752,126752,126752,2018-12-28,WBA,50.158054,69.080002,67.589996,68.559998,5181800.0,4.0,...,67.057912,46.443012,37.786170,-140.581480,57.411928,57.942806,57.250959,28.34,53.769986,3.0
126753,126753,126753,2018-12-28,WBD,24.639999,25.049999,24.400000,24.850000,3404300.0,4.0,...,31.365200,23.523799,36.637987,-143.645140,51.485596,28.545333,30.469833,28.34,53.769986,NaN
126754,126754,126754,2018-12-28,WDAY,159.559998,162.210007,154.149994,159.270004,1766000.0,4.0,...,173.266137,142.925862,55.732348,49.620993,19.241592,150.292333,141.290333,28.34,53.769986,1.0


In [3]:
def add_sentiment(train, sentiment, column_name='sentiment_deepseek'):
    sentiment_small = sentiment
    train_new = train
    train_new['date'] = pd.to_datetime(train_new['date'])

    # Step 2: Convert 'Date' to datetime without timezone
    sentiment_small['Date'] = pd.to_datetime(sentiment_small['Date']).dt.tz_localize(None)

    # Step 3: Rename 'Stock_symbol' to 'tic' and the sentiment column
    sentiment_small.rename(
        columns={'Stock_symbol': 'tic', column_name: 'llm_sentiment'},
        inplace=True
    )

    # Step 4: Merge train and sentiment_small into a new DataFrame
    train_new = train.merge(
        sentiment_small[['Date', 'tic', 'llm_sentiment']],
        left_on=['date', 'tic'],
        right_on=['Date', 'tic'],
        how='left'
    )

    # Step 5: Drop the additional 'Date' column
    train_new.drop(columns=['Date'], inplace=True)
    return train_new


In [ ]:
columns_to_load = ['Date', 'Stock_symbol', 'sentiment_deepseek']
sentiment = pd.read_csv('sentiment_deepseek.csv', usecols=columns_to_load)

In [ ]:
train_sentiment=add_sentiment(train,sentiment)

In [8]:
fundment = pd.read_csv("../Task_1_FinRL_DeepSeek_Stock/data/fundamental_state.csv")

/tmp/ipykernel_361390/955426586.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  fundment = pd.read_csv("../Task_1_FinRL_DeepSeek_Stock/data/fundamental_state.csv")


In [14]:
set(fundment["tic"].unique()) - set(train["tic"].unique())

{'AAPL.',
 'AEP.PA',
 'AEPPL',
 'AEPPZ',
 'AMD.',
 'AMZN.',
 'ASMLF',
 'AVGOP',
 'AZNCF',
 'CCZ',
 'CMCSK',
 'COST.',
 'CSCO.',
 'DISCB',
 'DISCK',
 'GMCR',
 'GOOG.',
 'HON.WI',
 'LLL.',
 'META.',
 'MSFT.',
 'NVDA.',
 'PE.PA',
 'PE.PB',
 'PE.PC',
 'PE.PD',
 'TMUSP',
 'TSLA.',
 'XCJ',
 'XEL.PA',
 'XEL.PB',
 'XEL.PC',
 'XEL.PD',
 'XEL.PE',
 'XEL.PG'}

In [28]:
fundment["gsector"].unique()

array([45, 20, 55, 35, 50, 30, 25, 10, 60])

In [6]:
import pandas as pd

In [19]:
train["tic"].nunique()

84

In [14]:
len(nasdaq_100_tickers_july_17_2023)

101

In [16]:
set(nasdaq_100_tickers_july_17_2023) - set(train["tic"].unique())

{'ABNB',
 'CEG',
 'CRWD',
 'DDOG',
 'GEHC',
 'GFS',
 'JD',
 'KHC',
 'LCID',
 'MRNA',
 'PDD',
 'PYPL',
 'SGEN',
 'TEAM',
 'TTD',
 'ZM',
 'ZS'}

In [15]:
set(nasdaq_100_tickers_july_17_2023) - imp_data

{'ABNB',
 'CEG',
 'CRWD',
 'DDOG',
 'GEHC',
 'GFS',
 'JD',
 'KHC',
 'LCID',
 'MRNA',
 'PDD',
 'PYPL',
 'SGEN',
 'TEAM',
 'TTD',
 'ZM',
 'ZS'}

In [16]:
# 파일 경로에 맞게 수정합니다.
file_path = '../Task_1_FinRL_DeepSeek_Stock/data/train-00000-of-00001.parquet'

# Parquet 파일 읽기 (pyarrow 또는 fastparquet이 필요)
df = pd.read_parquet(file_path)

In [20]:
df["Stock_symbol"].nunique()

89

In [23]:
len(imp_data)

84

In [24]:
df

,Unnamed: 0.1,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,0,12034.0,2023-12-16 18:00:00 UTC,Is FlexShares STOXX US ESG Select Index Fund (...,AAPL,https://www.nasdaq.com/articles/is-flexshares-...,None,NaN,"Making its debut on 07/13/2016, smart beta exc...","Taking into account individual holdings, Amazo...",Click to get this free report FlexShares STOXX...,Click to get this free report FlexShares STOXX...,"Taking into account individual holdings, Amazo..."
1,1,12068.0,2023-12-15 00:00:00 UTC,Guru Fundamental Report for AAPL,AAPL,https://www.nasdaq.com/articles/guru-fundament...,None,NaN,Below is Validea's guru fundamental report for...,APPLE INC (AAPL) is a large-cap growth stock i...,Detailed Analysis of APPLE INC AAPL Guru Analy...,Detailed Analysis of APPLE INC AAPL Guru Analy...,"Of the 22 guru strategies we follow, AAPL rate..."
2,2,12072.0,2023-12-14 00:00:00 UTC,2 Hot Warren Buffett Stocks That Raised Their ...,AAPL,https://www.nasdaq.com/articles/2-hot-warren-b...,None,NaN,The equity portfolio of Warren Buffett's inves...,Let's dig into the payout enhancements from th...,Let's dig into the payout enhancements from th...,Let's dig into the payout enhancements from th...,Let's dig into the payout enhancements from th...
3,3,12092.0,2023-12-13 00:00:00 UTC,"Technology Sector Update for 12/14/2023: ADBE,...",AAPL,https://www.nasdaq.com/articles/technology-sec...,None,NaN,Tech stocks were mixed Thursday afternoon with...,Apple (AAPL) and Alphabet's (GOOG) Google were...,Apple (AAPL) and Alphabet's (GOOG) Google were...,Apple (AAPL) and Alphabet's (GOOG) Google were...,Apple (AAPL) and Alphabet's (GOOG) Google were...
4,4,12138.0,2023-12-12 00:00:00 UTC,"Up 149% YTD, How High Can Roku (NASDAQ:ROKU) S...",AAPL,https://www.nasdaq.com/articles/up-149-ytd-how...,None,NaN,Media streaming company Roku’s (NASDAQ:ROKU) i...,Roku is also valued cheaper than its bigger co...,Roku is also valued cheaper than its bigger co...,Roku is also valued cheaper than its bigger co...,Roku is also valued cheaper than its bigger co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61822,61822,NaN,2019-02-17 00:00:00 UTC,"Spotlight On Walmart, Samsung And CAGNY (Stock...",XEL,https://seekingalpha.com/article/4241698-spotl...,Seeking Alpha,NaN,None,None,None,None,None
61823,61823,NaN,2019-02-16 00:00:00 UTC,"Stocks To Watch: Spotlight On Walmart, Samsung...",XEL,https://seekingalpha.com/article/4241661-stock...,Seeking Alpha,NaN,None,None,None,None,None
61824,61824,NaN,2019-02-15 00:00:00 UTC,ALLETE (ALE) Earnings and Revenues Beat Estima...,XEL,http://www.zacks.com/stock/news/355055/allete-...,Zacks,NaN,None,None,None,None,None
61825,61825,NaN,2019-01-24 00:00:00 UTC,Xcel Energy (XEL) Expected to Beat Earnings Es...,XEL,http://www.zacks.com/stock/news/349391/xcel-en...,Zacks,NaN,None,None,None,None,None
